## Tracking Using CSRT

In [8]:
import cv2
import csv
import os
import numpy as np

# Global variables for custom ROI selection
circle_center = None
circle_radius = 0
drawing = False

def select_circle(event, x, y, flags, param):
    """Mouse callback function for selecting a circular ROI."""
    global circle_center, circle_radius, drawing

    if event == cv2.EVENT_LBUTTONDOWN:
        # Start drawing
        circle_center = (x, y)
        circle_radius = 0
        drawing = True

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        # Update radius as the mouse moves
        circle_radius = int(np.sqrt((x - circle_center[0])**2 + (y - circle_center[1])**2))

    elif event == cv2.EVENT_LBUTTONUP:
        # Finish drawing
        drawing = False

def custom_roi_selection(frame, title="Select Object"):
    """Custom circular ROI selection."""
    global circle_center, circle_radius, drawing

    clone = frame.copy()
    cv2.namedWindow(title)
    cv2.setMouseCallback(title, select_circle)

    while True:
        temp_frame = clone.copy()
        if circle_center and circle_radius > 0:
            # Draw the circle as the user selects
            cv2.circle(temp_frame, circle_center, circle_radius, (0, 255, 0), 2)
        cv2.imshow(title, temp_frame)

        # Break on Enter key
        key = cv2.waitKey(1) & 0xFF
        if key == 13:  # Enter key
            break
        elif key == 27:  # Escape key
            circle_center, circle_radius = None, 0
            break

    cv2.destroyWindow(title)
    if circle_center and circle_radius > 0:
        # Convert circle to bounding box
        x = circle_center[0] - circle_radius
        y = circle_center[1] - circle_radius
        w = circle_radius * 2
        h = circle_radius * 2
        return (x, y, w, h)
    else:
        return None

def process_video(video_path, start_frame):
    """Process a single video and return tracking results, continuing frame count from start_frame."""
    video = cv2.VideoCapture(video_path)
    tracking_data = []

    if not video.isOpened():
        print(f"Error: Could not open video {video_path}.")
        return tracking_data, start_frame

    # Read the first frame
    success, frame = video.read()
    if not success:
        print(f"Error: Could not read video {video_path}.")
        return tracking_data, start_frame

    # Select ROIs manually using the custom selector
    print("Select Object 1:")
    bbox1 = custom_roi_selection(frame, "Select Object 1")
    if not bbox1:
        print("Selection canceled for Object 1.")
        return tracking_data, start_frame

    print("Select Object 2:")
    bbox2 = custom_roi_selection(frame, "Select Object 2")
    if not bbox2:
        print("Selection canceled for Object 2.")
        return tracking_data, start_frame

    # Initialize trackers for the selected objects
    tracker1 = cv2.TrackerCSRT_create()
    tracker2 = cv2.TrackerCSRT_create()
    tracker1.init(frame, bbox1)
    tracker2.init(frame, bbox2)

    frame_count = 0
    processed_frame_count = 0
    frame_step = 20

    while True:
        
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
        success, frame = video.read()
        if not success:
            print(f"End of video {video_path}.")
            break
            
        processed_frame_count += 1

        # Update trackers
        success1, bbox1 = tracker1.update(frame)
        success2, bbox2 = tracker2.update(frame)

        # Initialize coordinates as None
        x1, y1, x2, y2 = None, None, None, None

        if success1:
            (x1, y1, w1, h1) = [int(v) for v in bbox1]
            cv2.rectangle(frame, (x1, y1), (x1 + w1, y1 + h1), (255, 0, 0), 2)
            cv2.putText(frame, "Object 1", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        if success2:
            (x2, y2, w2, h2) = [int(v) for v in bbox2]
            cv2.rectangle(frame, (x2, y2), (x2 + w2, y2 + h2), (0, 255, 0), 2)
            cv2.putText(frame, "Object 2", (x2, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Store frame data
        tracking_data.append([
            frame_count,
            x1,
            y1,
            x2,
            y2
        ])

        # Display the frame
        cv2.imshow("Tracking", frame)

        frame_count += frame_step

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()

    return tracking_data, frame_count

## Single file processing

In [7]:
def process_single_file(input_file_path, output_csv):
    """Process a single video file and save to a CSV file."""
    # Verify file extension
    if not input_file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        raise ValueError("Unsupported file format. Please provide a video file (mp4, avi, mov, or mkv)")

    # Verify file exists
    if not os.path.exists(input_file_path):
        raise FileNotFoundError(f"Video file not found: {input_file_path}")

    print(f"Processing file: {input_file_path}")

    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    
    current_frame = 0

    # Create CSV file and write header
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(['Frame', 'Object1_X', 'Object1_Y', 'Object2_X', 'Object2_Y'])

        # Process the video
        video_tracking_data, current_frame = process_video(input_file_path, current_frame)

        # Write data for this video
        writer.writerows(video_tracking_data)

    print(f"Tracking results saved to {output_csv}")

# Example usage:
input_file = R"C:\Users\barei\Documents\GitHub\Ants\input videos\S6160004.MP4"
output_csv = R"C:\Users\barei\Documents\GitHub\Ants\Analyzed Data\2024_09_23\S6160004\coordinates.csv"
process_single_file(input_file, output_csv)

Processing file: C:\Users\barei\Documents\GitHub\Ants\input videos\S6160004.MP4



KeyboardInterrupt



### New Vesrion: Tracking Using Hugh Circles and Physical Constrains 

In [11]:
# Circle detection with visualization, no ant logic, supports frame skipping
import cv2
import numpy as np
import math
import os
import csv
import json
from statistics import median

# Utilities (same as original)
def resize_to_fit(frame, max_width=1280, max_height=720):
    h, w = frame.shape[:2]
    scale = min(max_width / w, max_height / h, 1.0)
    new_size = (int(w * scale), int(h * scale))
    return cv2.resize(frame, new_size), scale

def scale_point(p, scale):
    return (int(p[0] / scale), int(p[1] / scale))

def select_point(frame, title="Select point"):
    selected = {'pt': None}
    def callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            selected['pt'] = (x, y)
    cv2.namedWindow(title)
    cv2.setMouseCallback(title, callback)
    while True:
        temp = frame.copy()
        if selected['pt'] is not None:
            cv2.circle(temp, selected['pt'], 5, (0, 255, 0), -1)
        cv2.imshow(title, temp)
        key = cv2.waitKey(1) & 0xFF
        if key == 13 and selected['pt']: break
        elif key == 27: return None
    cv2.destroyWindow(title)
    return selected['pt']

def manual_circle_selection(frame, title="Select circle"):
    state = {'center': None, 'radius': 0, 'drawing': False}
    def callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            state['center'] = (x, y)
            state['drawing'] = True
        elif event == cv2.EVENT_MOUSEMOVE and state['drawing']:
            state['radius'] = int(math.hypot(x - state['center'][0], y - state['center'][1]))
        elif event == cv2.EVENT_LBUTTONUP:
            state['drawing'] = False
    cv2.namedWindow(title)
    cv2.setMouseCallback(title, callback)
    while True:
        temp = frame.copy()
        if state['center']:
            if state['radius'] > 0:
                cv2.circle(temp, state['center'], state['radius'], (0, 255, 0), 2)
        cv2.imshow(title, temp)
        key = cv2.waitKey(1) & 0xFF
        if key == 13 and state['center'] and state['radius'] > 0: break
        elif key == 27: return None, None
    cv2.destroyWindow(title)
    return state['center'], state['radius']

def track_objects(video_path, output_directory_path, frame_skipping=10):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video")
        return

    ret, first = cap.read()
    if not ret:
        print("Error reading first frame")
        return

    resized, scale = resize_to_fit(first)
    pivot1 = scale_point(select_point(resized, "Select Pivot 1"), scale)
    center1_disp, radius1_disp = manual_circle_selection(resized, "Select Circle 1")
    center1 = scale_point(center1_disp, scale)

    pivot2 = scale_point(select_point(resized, "Select Pivot 2"), scale)
    center2_disp, radius2_disp = manual_circle_selection(resized, "Select Circle 2")
    center2 = scale_point(center2_disp, scale)

    dist1 = int(math.hypot(center1[0] - pivot1[0], center1[1] - pivot1[1]))
    rad1 = int(radius1_disp / scale)
    dist2 = int(math.hypot(center2[0] - pivot2[0], center2[1] - pivot2[1]))
    rad2 = int(radius2_disp / scale)

    hmin = max(1, min(rad1, rad2) - 15)
    hmax = max(rad1, rad2) + 15

    # Output paths
    csv_path = os.path.join(output_directory_path, f"coordinates.csv")
    json_path = os.path.join(output_directory_path, f"metadata.json")

    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['frame', 'obj1_x', 'obj1_y', 'obj2_x', 'obj2_y'])

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        sm1, sm2 = None, None
        for i in range(0, total_frames, frame_skipping):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if not ret: break

            vis = frame.copy()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray_blur = cv2.GaussianBlur(gray, (7, 7), 0)
            circles = cv2.HoughCircles(gray_blur, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
                                       param1=100, param2=30,
                                       minRadius=hmin, maxRadius=hmax)
            best1 = best2 = None
            if circles is not None:
                for x, y, r in np.uint16(np.around(circles[0])):
                    d1 = abs(math.hypot(x - pivot1[0], y - pivot1[1]) - dist1)
                    d2 = abs(math.hypot(x - pivot2[0], y - pivot2[1]) - dist2)
                    if d1 < 20 and abs(r - rad1) < 15: best1 = (x, y)
                    if d2 < 20 and abs(r - rad2) < 15: best2 = (x, y)

            writer.writerow([i, *(best1 if best1 else ('', '')), *(best2 if best2 else ('', ''))])

            # Visualization
            for (pivot, center, color) in [(pivot1, best1, (255, 255, 0)), (pivot2, best2, (255, 0, 255))]:
                cv2.circle(vis, pivot, 5, (0, 0, 255), -1)
                cv2.circle(vis, pivot, dist1 if pivot == pivot1 else dist2, (0, 255, 0), 1)
                if center:
                    cv2.circle(vis, center, rad1 if pivot == pivot1 else rad2, color, 2)
                    cv2.line(vis, pivot, center, color, 1)

            cv2.putText(vis, f"Frame: {i}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            disp, _ = resize_to_fit(vis)
            cv2.imshow("Tracking", disp)
            if cv2.waitKey(1) & 0xFF == ord('q'): break

        cap.release()
    cv2.destroyAllWindows()

    # Post-process CSV to remove outliers and clean up
    with open(csv_path, 'r') as f:
        reader = list(csv.reader(f))
        header, rows = reader[0], reader[1:]

    clean_rows = []
    dists = []
    for i in range(1, len(rows)):
        prev = rows[i - 1]
        curr = rows[i]

        try:
            prev_coords = list(map(int, prev[1:5]))
            curr_coords = list(map(int, curr[1:5]))
        except ValueError:
            continue  # Skip incomplete or malformed rows

        dx1 = curr_coords[0] - prev_coords[0]
        dy1 = curr_coords[1] - prev_coords[1]
        dx2 = curr_coords[2] - prev_coords[2]
        dy2 = curr_coords[3] - prev_coords[3]

        dist = math.hypot(dx1, dy1) + math.hypot(dx2, dy2)
        dists.append((i, dist))  # Store index and distance

    # Calculate median distance
    dist_values = [d[1] for d in dists]
    if dist_values:
        median_dist = median(dist_values)
    else:
        median_dist = 0

    # Set threshold (e.g., 3x median)
    threshold = 3 * median_dist
    outlier_indices = set(idx for idx, d in dists if d > threshold)

    # Rebuild clean CSV
    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for i, row in enumerate(rows):
            if i in outlier_indices:
                continue
            if all(x.strip() != '' for x in row[1:5]):
                writer.writerow(row)

    # Save metadata
    meta = {
        'obj1_pivot': pivot1,
        'obj2_pivot': pivot2
    }
    with open(json_path, 'w') as f:
        json.dump(meta, f, indent=4)

    print("Done. Data cleaned and saved.")


# Example
track_objects(
    video_path=r"C:\Users\barei\Documents\GitHub\Ants\input videos\S6160004.MP4",
    output_directory_path=r"",
    frame_skipping=10
)


Done. Data cleaned and saved.
